In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm
import shutil
import re
import copy
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from pprint import pprint

In [ ]:
adr1 = "D:/PhD/Dataset/Lists/MajorNewsAgencies/us4.csv"
adr2 = "D:/PhD/Dataset/Lists/MajorNewsAgencies/india.csv"
adr3 = "D:/PhD/Dataset/Lists/MajorNewsAgencies/india-reliable.csv"
adr4 = "D:/PhD/Dataset/Lists/MajorNewsAgencies/us4-reliable.csv"

adr_fr_3days = "./DataCollection/Data/Florence _ 3days/R/"
adr_fnr_3days = "./DataCollection/Data/Florence _ 3days/NR/"

adr_fr_outlets = "./DataCollection/Data/Florence _ 3days/NR_credibleOutlets/"
adr_fr_outlets2 = "./DataCollection/Data/Florence _ 3days/NR_credibleOutlets2/"

In [ ]:
def tweetElements(tweet):
    rtFlag = True if "retweeted_status" in tweet.keys() else False
    qtFlag = True if "quoted_status" in tweet.keys() else False 

    if rtFlag == False and qtFlag == False:
        if tweet["truncated"] == True:
            text = copy.deepcopy(tweet["extended_tweet"]["full_text"])
            entities = copy.deepcopy(tweet["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(tweet["text"])
            entities = copy.deepcopy(tweet["entities"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user]
    
    elif rtFlag == True and qtFlag == False:
        rt = copy.deepcopy(tweet["retweeted_status"])
        if rt["truncated"] == True:
            text = copy.deepcopy(rt["extended_tweet"]["full_text"])
            entities = copy.deepcopy(rt["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(rt["text"])
            entities = copy.deepcopy(rt["entities"])
        rtUser = copy.deepcopy(rt["user"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user, rtUser]
    
    elif rtFlag == False and qtFlag == True:
        if tweet["truncated"] == True:
            text = copy.deepcopy(tweet["extended_tweet"]["full_text"])
            entities = copy.deepcopy(tweet["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(tweet["text"])
            entities = copy.deepcopy(tweet["entities"])
        user = copy.deepcopy(tweet["user"])
        qt = copy.deepcopy(tweet["quoted_status"])
        if qt["truncated"] == True:
            qtText = copy.deepcopy(qt["extended_tweet"]["full_text"])
            qtEntities = copy.deepcopy(qt["extended_tweet"]["entities"])
        else:
            qtText = copy.deepcopy(qt["text"])
            qtEntities = copy.deepcopy(qt["entities"])
        qtUser = copy.deepcopy(qt["user"])
        return [rtFlag, qtFlag, text, entities, user, qtText, qtEntities, qtUser]        
        
    elif rtFlag == True and qtFlag == True:
        rt = copy.deepcopy(tweet["retweeted_status"])
        qt = copy.deepcopy(tweet["quoted_status"])        
        if rt["truncated"] == True:
            text = copy.deepcopy(rt["extended_tweet"]["full_text"])
            entities = copy.deepcopy(rt["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(rt["text"])
            entities = copy.deepcopy(rt["entities"])
        rtUser = copy.deepcopy(rt["user"])        
        if qt["truncated"] == True:
            qtText = copy.deepcopy(qt["extended_tweet"]["full_text"])
            qtEntities = copy.deepcopy(qt["extended_tweet"]["entities"])
        else:
            qtText = copy.deepcopy(qt["text"])
            qtEntities = copy.deepcopy(qt["entities"])
        qtUser = copy.deepcopy(qt["user"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user, qtText, qtEntities, qtUser, rtUser]        

In [ ]:
for i in tqdm(os.listdir(adr_fr_outlets)):
    shutil.move(adr_fr_outlets+i, adr_fr_3days)

In [ ]:
df = pd.read_csv(adr1, delimiter=";", skiprows=1 , names = [1,2,3,4,5,6,7,8,9], encoding = "ISO-8859-1")
df = df.drop([1,5,6,7,8,9], axis="columns")
df = df.rename({2:"source", 3:"id", 4:"website"}, axis="columns")

df1 = df.drop(columns=["id"], axis="columns")
df2 = df.drop(columns=["website"], axis="columns")

df2["id"] = df2["id"].str.lower().str.replace("?", "").str.strip()

credibleWebsitesUsa = [i for i in df1.to_dict(orient="list")["website"] if str(i) != "nan"]
credibleAccountsUsa = [j for j in df2.to_dict(orient="list")["id"] if str(j) != "nan"]

In [ ]:
for file in tqdm(os.listdir(adr_fr_3days)):
    qtText, qtEntities, qtUser, rtUser, tweetText, qtTweetText, rtUser = None, None, None, None, None, None, None

    tweet = json.load(open(adr_fr_3days+file))    
    personalViewFlag = True
    
    elements = tweetElements(tweet)
    rtFlag = copy.deepcopy(elements[0])
    qtFlag = copy.deepcopy(elements[1]) 
    
    if rtFlag == False and qtFlag == False:
        text, entities, user = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4])
    elif rtFlag == True and qtFlag == False:
        text, entities, user, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5])
    elif rtFlag == False and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7])       
    elif rtFlag == True and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7]), copy.deepcopy(elements[8])
                    
    tweetText = text.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ")
    qtTweetText = qtText.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ") if qtFlag else " "
            
    urlCredibility = len([i for i in credibleWebsitesUsa if i in entities["urls"][0]["expanded_url"]]) > 0 if len(entities["urls"]) > 0 else False
    profileUrlCredibility = len([i for i in credibleWebsitesUsa if i in user["url"].lower()]) > 0 if user["url"] != None else False
    rtProfileUrlCredibility = (len([i for i in credibleWebsitesUsa if i in rtUser["url"].lower()]) > 0 if rtUser["url"] != None else False) if rtFlag else False
    misinfoFlag = re.search('.*(rumor|rumour|hoax|conspirac|fake-?news|fake-?info| myth | myths |disinfo|misinfo).*', tweetText) != None 
    factcheckFlag = re.search('.*(fact-?check|debunk).*', tweetText) != None   

In [ ]:
for file in tqdm(os.listdir(adr_fr_3days)):
    qtText, qtEntities, qtUser, rtUser, tweetText, qtTweetText, rtUser = None, None, None, None, None, None, None

    tweet = json.load(open(adr_fr_3days+file))    
    personalViewFlag = True
    
    elements = tweetElements(tweet)
    rtFlag = copy.deepcopy(elements[0])
    qtFlag = copy.deepcopy(elements[1]) 

    
    if rtFlag == False and qtFlag == False:
        text, entities, user = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4])
    elif rtFlag == True and qtFlag == False:
        text, entities, user, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5])
    elif rtFlag == False and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7])       
    elif rtFlag == True and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7]), copy.deepcopy(elements[8])
                    
    tweetText = text.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ")
    qtTweetText = qtText.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ") if qtFlag else " "
    
    lastDiscovery = False
    if re.search('.*puerto rican.*', tweetText, re.DOTALL) != None:
        lastDiscovery = True
    else:
        lastDiscovery = False  
       
    qlastDiscovery = False
    if re.search('.*puerto rican.*', qtTweetText, re.DOTALL) != None:
        qlastDiscovery = True
    else:
        qlastDiscovery = False  

        
    if (qlastDiscovery or lastDiscovery):
        shutil.move(adr_fr_3days+file, adr_fr_outlets)    